In [19]:
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings('ignore')

In [20]:
YEAR = ['2016', '2017', '2018', '2019', '2020']

# CSR

In [21]:
csr = {}
col = {}
for i in range(len(YEAR)):
    csr[i] = pd.read_excel('./社会责任评分/{}社会责任评分.xls'.format(YEAR[i]), dtype={"代码": str}).dropna().drop(
        columns=['评分等级', '相关链接', '所属地区', '行业类别'])
    csr[i]['代码'] = csr[i]['代码'].apply(lambda x: x[0:-3])
    col[i] = set(csr[i]['代码'])

common_index = list(col[0] & col[1] & col[2] & col[3] & col[4])
# print(csr[1])

In [22]:
processed_csr = {}
for i in range(len(YEAR)):
    processed_csr[i] = csr[i][csr[i]['代码'].isin(common_index)].sort_values('代码')
    processed_csr[i] = processed_csr[i][['代码', '总评分', '截止年份']]

In [23]:
panel_csr = processed_csr[0].append(processed_csr[1]).append(processed_csr[2]).append(processed_csr[3]).append(
    processed_csr[4])
panel_csr.rename(columns={'截止年份': '年份', '总评分': 'CSR'}, inplace=True)
print(panel_csr)
panel_csr.to_excel('./社会责任评分/panel_csr.xls', index=False)

          代码    CSR      年份
111   000001  69.45  2016.0
22    000002  76.52  2016.0
1957  000004  21.30  2016.0
2917  000005  16.33  2016.0
247   000006  63.26  2016.0
...      ...    ...     ...
2789  688368  18.87  2020.0
2803  688369  18.83  2020.0
2053  688388  22.21  2020.0
2480  688389  20.33  2020.0
2     688399  44.93  2020.0

[18080 rows x 3 columns]


# 经营绩效

In [24]:
new_bp = pd.read_excel('./盈利能力/FI_T5.xls', dtype={"Stkcd": str}).dropna()
new_bp["年份"] = new_bp["Accper"].apply(lambda x: x[0:-6]).astype(int)
new_bp["BP"] = new_bp["F050502B"] * 100
new_bp = new_bp.drop(columns=["Accper", "ShortName", "Typrep", "F050201B", "F050501B", "F050502B", "F051201B"]).rename(
    columns={"Stkcd": "代码"})
print(new_bp)

           代码    年份       BP
0      000002  2016  19.0279
1      000002  2017  21.3626
2      000002  2018  23.3355
3      000002  2019  21.7825
4      000002  2020  19.1154
...       ...   ...      ...
22502  900957  2016   0.7513
22503  900957  2017   7.3418
22504  900957  2018   6.3254
22505  900957  2019   5.4591
22506  900957  2020   4.4811

[17767 rows x 3 columns]


# 控制变量

## 资产负债率

In [25]:
debt = pd.read_excel('./控制变量/FI_T1.xls', dtype={"Stkcd": str}).dropna()
debt["年份"] = debt["Accper"].apply(lambda x: x[0:-6]).astype(int)
debt = debt.drop(columns=["Accper", "ShortName", "Typrep", "F010101A", "F010201A", "F011401A"]).rename(
    columns={"Stkcd": "代码", 'F011201A': 'LEV'})
print(debt)

           代码       LEV    年份
0      000002  0.805367  2016
1      000002  0.839813  2017
2      000002  0.845856  2018
3      000002  0.843590  2019
4      000002  0.812835  2020
...       ...       ...   ...
22502  900957  0.661961  2016
22503  900957  0.584235  2017
22504  900957  0.551612  2018
22505  900957  0.531432  2019
22506  900957  0.505506  2020

[22507 rows x 3 columns]


## 企业规模

In [26]:
size = pd.read_excel('./控制变量/PT_LCMAINFIN.xls', dtype={"Symbol": str}).dropna()
size["年份"] = size["EndDate"].apply(lambda x: x[0:-6]).astype(int)
size["企业规模"] = np.log(size["TotalAssets"])
size = size.drop(columns=["EndDate", "TotalAssets", "TotalLiability"]).rename(columns={"Symbol": "代码"})
print(size)

           代码    年份       企业规模
0      000002  2016  27.445504
1      000002  2017  27.784040
2      000002  2018  28.055360
3      000002  2019  28.179102
4      000002  2020  28.256519
...       ...   ...        ...
17884  900957  2016  20.876016
17885  900957  2017  20.742508
17886  900957  2018  20.730243
17887  900957  2019  20.740825
17888  900957  2020  20.731791

[17889 rows x 3 columns]


## 国有股持股比例

In [27]:
state = pd.read_excel('./控制变量/HLD_Capstru.xls', dtype={"Stkcd": str}).dropna()
state["年份"] = state["Reptdt"].apply(lambda x: x[0:-6]).astype(int)
state["国有股持股比例"] = state["Nshrstt"] / state["Nshrttl"]
state = state.drop(columns=["Reptdt", "Nshrstt", "Nshrttl"]).rename(columns={"Stkcd": "代码"})
print(state)

           代码    年份  国有股持股比例
0      000002  2016      0.0
1      000002  2017      0.0
2      000002  2018      0.0
3      000002  2019      0.0
4      000002  2020      0.0
...       ...   ...      ...
16939  900957  2016      0.0
16940  900957  2017      0.0
16941  900957  2018      0.0
16942  900957  2019      0.0
16943  900957  2020      0.0

[16944 rows x 3 columns]


## 托宾-Q

In [28]:
tobin = pd.read_excel('./控制变量/FI_T10.xls', dtype={"Stkcd": str}).dropna()
tobin["年份"] = tobin["Accper"].apply(lambda x: x[0:-6]).astype(int)
tobin = tobin.drop(columns=["Accper", "ShortName"]).rename(columns={"Stkcd": "代码", 'F100901A': 'TOBINQ'})
print(tobin)

           代码    TOBINQ    年份
0      000002  1.069010  2016
1      000002  1.117963  2017
2      000002  1.015736  2018
3      000002  1.046272  2019
4      000002  0.992654  2020
...       ...       ...   ...
16432  900957  2.057613  2016
16433  900957  1.923619  2017
16434  900957  1.536094  2018
16435  900957  1.454232  2019
16436  900957  1.338059  2020

[16437 rows x 3 columns]


## TOP10

In [29]:
top10 = pd.read_excel('./控制变量/EN_EquityNatureAll.xls', dtype={"Symbol": str}).dropna()
top10["年份"] = top10["EndDate"].apply(lambda x: x[0:-6]).astype(int)
top10["SOE_new"] = top10["EquityNatureID"].apply(lambda s: "1" if "1" in str(s) else "0")
top10 = top10.drop(columns=["EndDate", "ShortName", "EquityNature", "EquityNatureID"]).rename(
    columns={"Symbol": "代码"})
print(top10)

           代码  TopTenHoldersRate    年份 SOE_new
0      000002              57.40  2016       1
1      000002              68.53  2017       1
2      000002              71.70  2018       1
3      000002              64.24  2019       1
4      000002              58.98  2020       1
...       ...                ...   ...     ...
16938  900957              48.17  2016       0
16939  900957              48.43  2017       0
16940  900957              48.02  2018       0
16941  900957              48.17  2019       0
16942  900957              48.30  2020       0

[16543 rows x 4 columns]


## 行业性质

In [30]:
industry = pd.read_excel('./控制变量/STK_LISTEDCOINFOANL.xls', dtype={"Symbol": str}).dropna()
industry["年份"] = industry["EndDate"].apply(lambda x: x[0:-6]).astype(int)
industry["上市日期"] = industry["LISTINGDATE"].apply(lambda x: x[0:-6]).astype(int)
industry["行业性质"] = industry["IndustryCode"].apply(lambda s: "1" if "C" in str(s) else "0")
industry = industry.drop(
    columns=["ShortName", "IndustryName", "EndDate", "LISTINGDATE", "IndustryCode", "LISTINGSTATE"]).rename(
    columns={"Symbol": "代码"})
print(industry)

           代码    年份  上市日期 行业性质
0      000002  2016  1991    0
1      000002  2017  1991    0
2      000002  2018  1991    0
3      000002  2019  1991    0
4      000002  2020  1991    0
...       ...   ...   ...  ...
17328  900957  2016  2000    0
17329  900957  2017  2000    0
17330  900957  2018  2000    0
17331  900957  2019  2000    0
17332  900957  2020  2000    0

[17333 rows x 4 columns]


# 汇总数据

In [31]:
# 合并所有数据
panel_raw_data = new_bp
panel_raw_data = pd.merge(panel_raw_data, panel_csr, on=["代码", "年份"], how='inner')
panel_raw_data = pd.merge(panel_raw_data, top10, on=["代码", "年份"], how='inner')
panel_raw_data = pd.merge(panel_raw_data, tobin, on=["代码", "年份"], how='inner')
panel_raw_data = pd.merge(panel_raw_data, debt, on=["代码", "年份"], how='inner')
panel_raw_data = pd.merge(panel_raw_data, size, on=["代码", "年份"], how='inner')
panel_raw_data = pd.merge(panel_raw_data, state, on=["代码", "年份"], how='inner')
panel_raw_data = pd.merge(panel_raw_data, industry, on=["代码", "年份"], how='inner')
print(panel_raw_data)

           代码    年份       BP    CSR  TopTenHoldersRate SOE_new    TOBINQ  \
0      000002  2016  19.0279  76.52              57.40       1  1.069010   
1      000002  2017  21.3626  41.87              68.53       1  1.117963   
2      000002  2018  23.3355  41.88              71.70       1  1.015736   
3      000002  2019  21.7825  41.80              64.24       1  1.046272   
4      000002  2020  19.1154  41.74              58.98       1  0.992654   
...       ...   ...      ...    ...                ...     ...       ...   
14984  688388  2020   7.2075  22.21              48.61       0  5.316332   
14985  688389  2019  11.7062  20.79              71.34       0  1.410722   
14986  688389  2020  13.1285  20.33              70.40       0  3.199495   
14987  688399  2019  12.4555  23.56              60.04       0  1.467813   
14988  688399  2020  61.0413  44.93              57.82       0  3.428711   

            LEV       企业规模   国有股持股比例  上市日期 行业性质  
0      0.805367  27.445504  0.000000 

In [32]:
balance_index = panel_raw_data.groupby(by='代码')['年份'].count()[
    panel_raw_data.groupby(by='代码')['年份'].count() == 5].index
balance_data = panel_raw_data[panel_raw_data['代码'].isin(balance_index)]

至此得到平衡面板数据balance_data

下一步去除：
1. 若数据缺失或有异常，则予以剔除
2. 上市时间小于5的公司,为了避免IPO效应带来的影响，剔除2016年及之后上市的公司样本
3. 所有“金融类”公司，鉴于金融行业具有普遍高负债等行业特性，剔除所有金融类公司
4. 连续亏损的公司所存在的问题可能对论文研究造成一定影响，故剔除所有ST、SST、PT、\*ST、S\*ST公司

## 删除2016年及之后上市的公司

In [33]:
final_data = balance_data[balance_data['上市日期'] < 2016]

In [34]:
# 将代码、行业性质、股权性质转化为int类型，便于Stata识别
final_data['IND'] = final_data['行业性质'].astype(int)
# 改正performance,负债水平,股权性质
final_data = final_data.drop(columns=['行业性质', '上市日期']).rename(
    columns={'代码': 'Stk_cd', '国有股持股比例': 'SOE', '企业规模': 'SIZE',
             '年份': 'YEAR', 'TopTenHoldersRate': 'TOP10'})
final_data.to_excel('./final_data.xls', index=False)
# print(final_data)

## 检查是否有异常值或缺失值

In [35]:
# 检查是否平衡
imbalance_index = final_data.groupby(by='Stk_cd')['YEAR'].count()[
    final_data.groupby(by='Stk_cd')['YEAR'].count() != 5].index
print(imbalance_index)
# 检查是否有缺失值
if final_data.isnull().values.any():
    print("有缺失值")
else:
    print("无缺失值")

# final_data[final_data.isnull().values==True]

Index([], dtype='object', name='Stk_cd')
无缺失值


In [36]:
print(final_data)

       Stk_cd  YEAR       BP    CSR  TOP10 SOE_new    TOBINQ       LEV  \
0      000002  2016  19.0279  76.52  57.40       1  1.069010  0.805367   
1      000002  2017  21.3626  41.87  68.53       1  1.117963  0.839813   
2      000002  2018  23.3355  41.88  71.70       1  1.015736  0.845856   
3      000002  2019  21.7825  41.80  64.24       1  1.046272  0.843590   
4      000002  2020  19.1154  41.74  58.98       1  0.992654  0.812835   
...       ...   ...      ...    ...    ...     ...       ...       ...   
14850  603999  2016   4.8016  20.06  71.33       1  2.393348  0.124609   
14851  603999  2017   4.1547  17.13  68.59       1  1.541950  0.121880   
14852  603999  2018   2.3118  15.26  68.83       1  1.528609  0.136092   
14853  603999  2019   3.6033  15.45  64.51       1  2.210450  0.151742   
14854  603999  2020   4.2531  16.20  64.62       1  1.654336  0.166607   

            SIZE  SOE  IND  
0      27.445504  0.0    0  
1      27.784040  0.0    0  
2      28.055360  0.0   